<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao21_Word2VecLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de Texto com Word2Vec (Gensim) e LSTM (Keras)

Este notebook demonstra uma abordagem de "Transfer Learning" para NLP. Em vez de treinar uma camada de Embedding do zero, vamos usar vetores de palavras (word embeddings) pré-treinados com o algoritmo **Word2Vec**.

O fluxo será:
1. Carregar e pré-processar os dados de texto.
2. Treinar nosso próprio modelo Word2Vec (via `gensim`) nos textos.
3. Preparar os textos para o Keras (Tokenização e Padding).
4. Criar uma Matriz de Embedding que "injeta" os vetores do Word2Vec no Keras.
5. Construir, treinar e avaliar um modelo LSTM que utiliza esses embeddings congelados.

In [1]:
# Célula 1: Instalação e Importações

# O Gensim é a biblioteca padrão para modelagem de tópicos e Word2Vec
!pip install gensim

import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
import re
import warnings

# Suprimir avisos
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 12.2 MB/s eta 0:00:00


## Passo 1: Carregar e Preparar Dados

Primeiro, carregamos o mesmo conjunto de dados (20 Newsgroups, 2 categorias) da comparação anterior.

In [2]:
# Célula 2: Carregar os dados
categorias = ['comp.graphics', 'sci.crypt']
dados = fetch_20newsgroups(subset='all', categories=categorias, shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))

# Separar dados e rótulos
textos = dados.data
rotulos = dados.target # 0 para 'comp.graphics', 1 para 'sci.crypt'

# Dividir em treino e teste
X_train_text, X_test_text, y_train, y_test = train_test_split(textos, rotulos, test_size=0.2, random_state=42)

print(f"Amostras de treino: {len(X_train_text)}")
print(f"Amostras de teste: {len(X_test_text)}")

Amostras de treino: 1571
Amostras de teste: 393


## Passo 2: Treinar o Modelo Word2Vec (Gensim)

Para treinar o Word2Vec, precisamos de sentenças tokenizadas (listas de palavras). Vamos fazer um pré-processamento simples (remover não-alfanuméricos, passar para minúsculas) e treinar o modelo em **todos** os textos (treino e teste), pois o Word2Vec é um aprendizado não supervisionado e se beneficia de mais dados.

In [4]:
# Célula 3: Pré-processamento para o Gensim

def preprocess_text_gensim(text):
    text = re.sub(r'\W+', ' ', text) # Remove caracteres não-alfanuméricos
    text = text.lower()
    return text.split() # Retorna uma lista de palavras (tokens)

# Processar todos os textos (treino + teste)
textos_completos = X_train_text + X_test_text
textos_tokenizados = [preprocess_text_gensim(doc) for doc in textos_completos]

print(f"Primeiro documento tokenizado (para Gensim):\n{textos_tokenizados[0][:20]}...")

Primeiro documento tokenizado (para Gensim):
[]...


In [5]:
# Célula 4: Treinar o modelo Word2Vec

# Parâmetros do Word2Vec
DIM_EMBEDDING = 100    # Dimensão dos vetores de palavras
WINDOW_SIZE = 5      # Tamanho da janela de contexto
MIN_COUNT = 3        # Ignorar palavras com frequência menor que esta
WORKERS = 4          # Threads para paralelizar o treino

print("Treinando modelo Word2Vec...")
w2v_model = Word2Vec(
    sentences=textos_tokenizados,
    vector_size=DIM_EMBEDDING,
    window=WINDOW_SIZE,
    min_count=MIN_COUNT,
    workers=WORKERS
)

print("Treino do Word2Vec concluído.")

# Verificar o vocabulário aprendido
vocab_size_w2v = len(w2v_model.wv.key_to_index)
print(f"Tamanho do vocabulário do Word2Vec (com min_count={MIN_COUNT}): {vocab_size_w2v} palavras")

# Testar o modelo Word2Vec
try:
    print("\nPalavras mais similares a 'computer':")
    print(w2v_model.wv.most_similar('computer'))
except KeyError:
    print("'computer' não está no vocabulário (provavelmente filtrado pelo min_count)")

try:
    print("\nPalavras mais similares a 'crypto':")
    print(w2v_model.wv.most_similar('crypto'))
except KeyError:
    print("'crypto' não está no vocabulário")

Treinando modelo Word2Vec...
Treino do Word2Vec concluído.
Tamanho do vocabulário do Word2Vec (com min_count=3): 9871 palavras

Palavras mais similares a 'computer':
[('visualisation', 0.9082591533660889), ('research', 0.8919011354446411), ('symposium', 0.8791888356208801), ('aided', 0.875365674495697), ('primitives', 0.8682764172554016), ('atmospheric', 0.8668799996376038), ('systems', 0.8644639849662781), ('itti', 0.8635594248771667), ('visualization', 0.8622421026229858), ('national', 0.8614664077758789)]

Palavras mais similares a 'crypto':
[('strong', 0.9829712510108948), ('being', 0.9718181490898132), ('classified', 0.9706794023513794), ('legal', 0.9664202332496643), ('clinton', 0.9659008979797363), ('door', 0.9649192094802856), ('phones', 0.9636639356613159), ('cripple', 0.9628631472587585), ('fact', 0.9614049792289734), ('illegal', 0.9572980403900146)]


## Passo 3: Preparar Dados para o LSTM (Keras)

Agora, fazemos o processo padrão do Keras: tokenizar e padronizar (padding) as sequências. É **crucial** que o `Tokenizer` do Keras seja ajustado *apenas* nos dados de treino para evitar vazamento de dados (data leakage).

In [6]:
# Célula 5: Tokenização e Padding do Keras

# Parâmetros do Keras
MAX_PALAVRAS_VOCAB = 10000 # Tamanho máximo do vocabulário para o Keras
MAX_COMPRIMENTO_SEQ = 250  # Comprimento máximo de cada documento (o mesmo de antes)

# 1. Tokenizar (usando Keras)
tokenizer = Tokenizer(num_words=MAX_PALAVRAS_VOCAB, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text) # Ajustar SOMENTE no treino

# Converter textos em sequências de inteiros
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq = tokenizer.texts_to_sequences(X_test_text)

# 2. Padronizar (Padding)
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_COMPRIMENTO_SEQ, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_COMPRIMENTO_SEQ, padding='post', truncating='post')

# Guardar o índice de palavras do Keras
word_index = tokenizer.word_index
# O vocabulário real do Keras será o min(MAX_PALAVRAS_VOCAB, len(word_index)) + 1 (para o padding)
vocab_size_keras = min(MAX_PALAVRAS_VOCAB, len(word_index) + 1)

print(f"Tamanho do vocabulário do Keras: {vocab_size_keras}")
print(f"Formato dos dados de treino padronizados: {X_train_pad.shape}")

Tamanho do vocabulário do Keras: 10000
Formato dos dados de treino padronizados: (1571, 250)


## Passo 4: Criar a Matriz de Embedding (A "Ponte")

Este é o passo crucial.

Vamos criar uma matriz onde `matriz[i]` conterá o vetor Word2Vec para a palavra de índice `i` no `Tokenizer` do Keras. Se uma palavra do Keras não existir no nosso modelo Word2Vec (talvez por ter sido filtrada pelo `min_count`), seu vetor permanecerá como zeros.

In [7]:
# Célula 6: Construção da Matriz de Embedding

# Inicializar uma matriz de zeros
# Usamos 'vocab_size_keras' que já considera o limite MAX_PALAVRAS_VOCAB
embedding_matrix = np.zeros((vocab_size_keras, DIM_EMBEDDING))

palavras_encontradas = 0
palavras_nao_encontradas = 0

for word, i in word_index.items():
    if i >= MAX_PALAVRAS_VOCAB: # Não ultrapassar o limite de vocabulário
        continue

    if word in w2v_model.wv: # Se a palavra existe no modelo Word2Vec
        embedding_matrix[i] = w2v_model.wv[word] # Copia o vetor
        palavras_encontradas += 1
    else:
        palavras_nao_encontradas += 1 # A palavra fica com vetor de zeros

print(f"Matriz de Embedding criada com formato: {embedding_matrix.shape}")
print(f"{palavras_encontradas} palavras do Keras foram encontradas no Word2Vec.")
print(f"{palavras_nao_encontradas} palavras não foram encontradas (ficarão como vetores de zero).")

Matriz de Embedding criada com formato: (10000, 100)
9071 palavras do Keras foram encontradas no Word2Vec.
928 palavras não foram encontradas (ficarão como vetores de zero).


## Passo 5: Construir o Modelo LSTM

Agora, construímos o modelo LSTM. A diferença está na camada `Embedding`:
1.  Passamos `weights=[embedding_matrix]` para carregar nossos pesos pré-treinados.
2.  Definimos `trainable=False` para "congelar" esses pesos. O modelo só treinará as camadas LSTM e Densa.

In [12]:
# Célula 7: Definindo o modelo LSTM com embeddings pré-treinados

modelo_w2v_lstm = Sequential(name="Modelo_Word2Vec_LSTM")

# Camada de Embedding - A GRANDE MUDANÇA ESTÁ AQUI
modelo_w2v_lstm.add(Embedding(
    input_dim=vocab_size_keras,     # Tamanho do vocabulário do Keras
    output_dim=DIM_EMBEDDING,       # Dimensão dos vetores (do Word2Vec)
    input_length=MAX_COMPRIMENTO_SEQ,
    weights=[embedding_matrix],     # Carrega a matriz pré-treinada
    trainable=False                 # CONGELA a camada de embedding
))

modelo_w2v_lstm.add(SpatialDropout1D(0.2))
modelo_w2v_lstm.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
modelo_w2v_lstm.add(Dense(16, activation='relu'))
modelo_w2v_lstm.add(Dense(1, activation='sigmoid'))

modelo_w2v_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Estrutura do Modelo:")
modelo_w2v_lstm.summary()

Estrutura do Modelo:


Model: "Modelo_Word2Vec_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_2             │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,000,000 (3.81 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,000,000 (3.81 MB)

## Passo 6: Treinar e Avaliar

Finalmente, treinamos e avaliamos o modelo. O processo de treino deve ser mais rápido, pois a camada de Embedding (que geralmente tem muitos parâmetros) não está sendo treinada.

In [15]:
# Célula 8: Treinamento e Avaliação

# Converter rótulos para numpy array
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

print("\nTreinando o modelo W2V+LSTM...")

historico = modelo_w2v_lstm.fit(
    X_train_pad,
    y_train_np,
    epochs=100, # Treinar por mais épocas, já que o embedding está congelado
    batch_size=32,
    validation_data=(X_test_pad, y_test_np),
    verbose=2
)

# Avaliação final
loss, acuracia_w2v_lstm = modelo_w2v_lstm.evaluate(X_test_pad, y_test_np, verbose=0)

print(f"\nAcurácia (Word2Vec + LSTM): {acuracia_w2v_lstm:.4f}")


Treinando o modelo W2V+LSTM...
Epoch 1/100
50/50 - 16s - 310ms/step - accuracy: 0.8377 - loss: 0.3669 - val_accuracy: 0.8321 - val_loss: 0.4052
Epoch 2/100
50/50 - 15s - 298ms/step - accuracy: 0.8370 - loss: 0.3805 - val_accuracy: 0.8244 - val_loss: 0.3748
Epoch 3/100
50/50 - 15s - 301ms/step - accuracy: 0.8491 - loss: 0.3547 - val_accuracy: 0.8372 - val_loss: 0.3680
Epoch 4/100
50/50 - 20s - 405ms/step - accuracy: 0.8523 - loss: 0.3490 - val_accuracy: 0.8295 - val_loss: 0.3819
Epoch 5/100
50/50 - 20s - 399ms/step - accuracy: 0.8555 - loss: 0.3383 - val_accuracy: 0.8346 - val_loss: 0.3642
Epoch 6/100
50/50 - 14s - 285ms/step - accuracy: 0.8536 - loss: 0.3347 - val_accuracy: 0.8244 - val_loss: 0.3631
Epoch 7/100
50/50 - 21s - 414ms/step - accuracy: 0.8644 - loss: 0.3155 - val_accuracy: 0.8321 - val_loss: 0.3630
Epoch 8/100
50/50 - 21s - 416ms/step - accuracy: 0.8593 - loss: 0.3155 - val_accuracy: 0.8270 - val_loss: 0.3559
Epoch 9/100
50/50 - 14s - 285ms/step - accuracy: 0.8612 - loss: 

## Conclusão

Esta abordagem nos permite usar o "conhecimento" semântico capturado pelo Word2Vec (treinado de forma não supervisionada) como ponto de partida para um classificador supervisionado (LSTM).

**Vantagens:**
* **Eficiência de Treino:** Congelar a camada de Embedding reduz drasticamente o número de parâmetros treináveis, tornando o treino do LSTM mais rápido.
* **Melhor Desempenho em Dados Pequenos:** Se tivéssemos poucos dados de treino *rotulados*, usar embeddings treinados em um corpus muito maior (não supervisionado) traria uma grande vantagem de generalização.